In [ ]:
from utils import *
res_1km = 1 / 12 / 10
res_500m = res_1km / 2

import warnings
warnings.filterwarnings("ignore")
from rasterio.enums import Resampling

In [ ]:
path_conflict_data = Path("E:/GEODATA/Raw Data/war data")
df_conflict = pd.read_csv(path_conflict_data / "2000-01-01-2023-12-31.csv")

res = 1 / 12 / 10 / 2
x_template, y_template = np.arange(-180 + res / 2, 180, res), np.arange(-60 + res / 2 , 85, res)
x_bins, y_bins = np.arange(-180, 180+res/2, res), np.arange(-60, 85+res/2, res)

df_conflict["x"] = pd.cut(df_conflict["longitude"], x_bins, labels=x_template).astype(float)
df_conflict["y"] = pd.cut(df_conflict["latitude"], y_bins, labels=y_template).astype(float)

In [ ]:
df_conflict_years = df_conflict[["year", "event_type", "x", "y"]].drop_duplicates()

In [ ]:
gdf_conflict_years = gpd.GeoDataFrame(df_conflict_years, geometry=gpd.points_from_xy(df_conflict_years["x"], df_conflict_years["y"], crs="epsg:4326"))
gdf_conflict_years["idx"] = np.arange(gdf_conflict_years.shape[0])
gdf_conflict_years_buffer = gdf_conflict_years.copy()
gdf_conflict_years_buffer["geometry"] = gdf_conflict_years_buffer.buffer(res_500m * 2 * 20)

In [ ]:
gdf_conflict_years_buffer_dissolve = gdf_conflict_years_buffer.dissolve("year").reset_index()

In [ ]:
da_luc_500m = xr.open_dataarray(path_data / "luc/2001-2023-500m.tif").sel(band=slice(2, 24))
da_luc_500m["band"] = np.arange(2001, 2024)
da_luc_500m = da_luc_500m.rename({"band": "year"})
da_luc_500m.attrs = {}

In [ ]:
st1 = """Value	Color	Description
1	#05450a	Evergreen Needleleaf Forests: dominated by evergreen conifer trees (canopy >2m). Tree cover >60%.
2	#086a10	Evergreen Broadleaf Forests: dominated by evergreen broadleaf and palmate trees (canopy >2m). Tree cover >60%.
3	#54a708	Deciduous Needleleaf Forests: dominated by deciduous needleleaf (larch) trees (canopy >2m). Tree cover >60%.
4	#78d203	Deciduous Broadleaf Forests: dominated by deciduous broadleaf trees (canopy >2m). Tree cover >60%.
5	#009900	Mixed Forests: dominated by neither deciduous nor evergreen (40-60% of each) tree type (canopy >2m). Tree cover >60%.
6	#c6b044	Closed Shrublands: dominated by woody perennials (1-2m height) >60% cover.
7	#dcd159	Open Shrublands: dominated by woody perennials (1-2m height) 10-60% cover.
8	#dade48	Woody Savannas: tree cover 30-60% (canopy >2m).
9	#fbff13	Savannas: tree cover 10-30% (canopy >2m).
10	#b6ff05	Grasslands: dominated by herbaceous annuals (<2m).
11	#27ff87	Permanent Wetlands: permanently inundated lands with 30-60% water cover and >10% vegetated cover.
12	#c24f44	Croplands: at least 60% of area is cultivated cropland.
13	#a5a5a5	Urban and Built-up Lands: at least 30% impervious surface area including building materials, asphalt and vehicles.
14	#ff6d4c	Cropland/Natural Vegetation Mosaics: mosaics of small-scale cultivation 40-60% with natural tree, shrub, or herbaceous vegetation.
15	#69fff8	Permanent Snow and Ice: at least 60% of area is covered by snow and ice for at least 10 months of the year.
16	#f9ffa4	Barren: at least 60% of area is non-vegetated barren (sand, rock, soil) areas with less than 10% vegetation.
17	#1c0dff	Water Bodies: at least 60% of area is covered by permanent water bodies."""

class_table = pd.read_csv(StringIO(st1), sep="\t")
class_table["Value"] = class_table["Value"].astype(int)
class_table["luc"] = class_table["Description"].str.split(":", expand=True)[0]

code2color = class_table.set_index("Value")["Color"].to_dict()
code2luc = class_table.set_index("Value")["luc"].to_dict()

In [ ]:
def cal_luc_count_1year(year):
    da_luc_conflict_1year = da_luc_500m.sel(year=year).rio.clip(gdf_conflict_years_buffer_dissolve.query("year == @year").geometry)
    ar_luc_code, ar_luc_count = np.unique(da_luc_conflict_1year.values, return_counts=True)

    df_luc_count_1year = pd.DataFrame({"luc": ar_luc_code, "count": ar_luc_count}).dropna()\
        .assign(color=lambda x: x["luc"].map(code2color))\
        .assign(luc=lambda x: x["luc"].map(code2luc))\
        .assign(area=lambda _df: _df["count"] * 500 **2)\
        .assign(year=year)
    return df_luc_count_1year

In [ ]:
for year in range(2001, 2024):
    if (path_out := path_data / f"reanalysis500m/conflict_luc_statistic/luc_count_{year}.csv").exists():
        continue
    df_luc_count_1year = cal_luc_count_1year(year)
    df_luc_count_1year.to_csv(path_out, index=False)